In [1]:
from bitstring import BitArray, BitStream
from h264bits import H264Bits

In [13]:
from nalsps import SPS
from nalpps import PPS
from nalslice import Slice

In [2]:
input_file = open("test.264", "rb")
nalus_ba = list(BitArray(input_file).split('0x000001', bytealigned=True))[1:]

In [14]:
sps = None
ppss = []
slices = []

In [3]:
for nalu_ba in nalus_ba:
    print(nalu_ba)
    # enulation_prevention_three_byte
    nalu_ba.replace('0x000003', '0x0000', bytealigned=True)
    nalu_bs = BitStream(nalu_ba)
    nb = H264Bits(nalu_bs)
    params = {"forbidden_zero_bit" : nb.f(1),
              "nal_ref_idc" : nb.u(2),
              "nal_unit_type" : nb.u(5)}
    print(params)
    if params["nal_unit_type"] == 7: # SPS
        sps = SPS(nb, params = params)
        # dump_params(sps, "sps.json")
    elif params["nal_unit_type"] == 8: # PPS
        pps = PPS(nb, sps = sps, params = params)
        ppss.append(pps)
        fname = "pps_" + str(len(ppss)) + ".json"
        # dump_params(pps, fname)
    elif params["nal_unit_type"] in [1, 5]: # Slice
        slice = Slice(nb, sps = sps, ppss = ppss, params = params)
        # decode_slice(slice)
        break
        slices.append(slice)
        dump_mbs(slice, "slice_" + str(len(slices)) + "_mb.json")
        fname = "slice_" + str(len(slices)) + ".json"
        dump_params(slice, fname)
    else:
        print("Unknown Slice type, ignore...")


0x00000167640028acd182c4e400
{'forbidden_zero_bit': 0, 'nal_ref_idc': 3, 'nal_unit_type': 7}
0x00000168e94a38b000
{'forbidden_zero_bit': 0, 'nal_ref_idc': 3, 'nal_unit_type': 8}
0x000001685a52de2c00
{'forbidden_zero_bit': 0, 'nal_ref_idc': 3, 'nal_unit_type': 8}
0x000001687a52ee2c00
{'forbidden_zero_bit': 0, 'nal_ref_idc': 3, 'nal_unit_type': 8}
0x000001658884013fea9ec2cecbaa919dadf9de624772b20dcf7c4132264f663261a1d036ed43718133e5cb93b219306306ba3284d854cfb5dd10d07b6028516da42572c04e58a03ee73b6f348e461f89e608d0941a47785eca67b97e5b38227a624d28e88098b062c099393d9353a0667f7852bcb7e727b587798be96d...
{'forbidden_zero_bit': 0, 'nal_ref_idc': 3, 'nal_unit_type': 5}
0x000001419a24c93fe8a7caeab8fdaf05736d2bba35e76145d182df4f9f85b98689e72e9653194c000ea9a62d7190958fb12df83301e283550e29c50791ce74bc5fd1ae6eee0eb9048f2ba0fe3df5053631a822a04e18dae5c57a0be040cd3a8455f48905fe2742e3508af99eeff4cee3a40338d3ea610c98ab0862ae2d...
{'forbidden_zero_bit': 0, 'nal_ref_idc': 2, 'nal_unit_type': 1}
0x000001019e

The `forbidden_zero_bit` is always set to zero and is used to check for bit errors or synchronization issues. If this bit is not zero, it indicates an error in the stream.

The `nal_ref_idc` field in the NALU (Network Abstraction Layer Unit) header in H.264 video coding is a 2-bit field. Therefore, it can take on the following four possible values:

- 0: The NALU does not contain a sequence or picture parameter set, slice of a reference picture, or a slice data partition of a reference picture.
- 1: The NALU contains a slice of a reference picture or a slice data partition of a reference picture, but not a sequence or picture parameter set.
- 2: The NALU contains a sequence or picture parameter set, but not a slice of a reference picture or a slice data partition of a reference picture.
- 3: The NALU contains a sequence or picture parameter set, a slice of a reference picture, or a slice data partition of a reference picture.

These values are used to indicate the importance of the NALU in terms of error recovery and the decoding process. Higher values indicate higher importance.

In H.264 video coding, the `nal_unit_type` field in the NALU (Network Abstraction Layer Unit) header specifies the type of RBSP (Raw Byte Sequence Payload) data structure contained in the NALU. Here are the possible NAL unit types:

- 0: Unspecified
- 1: Coded slice of a non-IDR picture
- 2: Coded slice data partition A
- 3: Coded slice data partition B
- 4: Coded slice data partition C
- 5: Coded slice of an IDR picture
- 6: Supplemental enhancement information (SEI)
- 7: Sequence parameter set
- 8: Picture parameter set
- 9: Access unit delimiter
- 10: End of sequence
- 11: End of stream
- 12: Filler data
- 13: Sequence parameter set extension
- 14: Prefix NAL unit
- 15: Subset sequence parameter set
- 16-18: Reserved
- 19: Coded slice of an auxiliary coded picture without partitioning
- 20: Coded slice extension
- 21: Coded slice extension for depth view components
- 22-23: Reserved
- 24: STAP-A Single-time aggregation packet
- 25: STAP-B Single-time aggregation packet
- 26: MTAP16 Multi-time aggregation packet
- 27: MTAP24 Multi-time aggregation packet
- 28: FU-A Fragmentation unit
- 29: FU-B Fragmentation unit
- 30-31: Unspecified

Each of these types corresponds to a different kind of data or operation in the H.264 video stream.

In [ ]:
print(sps)

In [9]:
# SPS
nalu_ba = nalus_ba[0]
print(nalu_ba)
nalu_ba.replace('0x000003', '0x0000', bytealigned=True)
nalu_bs = BitStream(nalu_ba)
nb = H264Bits(nalu_bs)
params = {"forbidden_zero_bit" : nb.f(1),
            "nal_ref_idc" : nb.u(2),
            "nal_unit_type" : nb.u(5)}
print(params)

0x00000167640028acd182c4e400
{'forbidden_zero_bit': 0, 'nal_ref_idc': 3, 'nal_unit_type': 7}


In [10]:
# IDR
nalu_ba = nalus_ba[4]
print(nalu_ba)
nalu_ba.replace('0x000003', '0x0000', bytealigned=True)
nalu_bs = BitStream(nalu_ba)
nb = H264Bits(nalu_bs)
params = {"forbidden_zero_bit" : nb.f(1),
            "nal_ref_idc" : nb.u(2),
            "nal_unit_type" : nb.u(5)}
print(params)

0x000001658884013fea9ec2cecbaa919dadf9de624772b20dcf7c4132264f663261a1d036ed43718133e5cb93b219306306ba3284d854cfb5dd10d07b6028516da42572c04e58a03ee73b6f348e461f89e608d0941a47785eca67b97e5b38227a624d28e88098b062c099393d9353a0667f7852bcb7e727b587798be96d...
{'forbidden_zero_bit': 0, 'nal_ref_idc': 3, 'nal_unit_type': 5}
